In [1]:
import requests
import json
from bs4 import BeautifulSoup 
import re

In [ ]:
#find all ITIS employees from kpfu site
body = {
    ('p_sub', (None, 7860)),
    ('p_id', (None, 10857)),
    ('p_rec_count', (None, 0))
}
url = "https://kpfu.ru/main_page"
response = requests.post(url, files=body)

page = BeautifulSoup(response.text, 'lxml')
count = page.find('div', id='ss_content').find_all('div', recursive=False)[1].find('table').text
count = int(re.search(r'\d+', re.search(r'НАЙДЕНО \d+ СОТРУДНИКОВ', count)[0])[0])+5

body = {
    ('p_sub', (None, 7860)),
    ('p_id', (None, 10857)),
    ('p_rec_count', (None, count)),
    ('p_order', (None, 1)),
}
url = "https://kpfu.ru/main_page"
response = requests.post(url, files=body)

page = BeautifulSoup(response.text, 'lxml')
persons = page.find('div', id='ss_content').find_all('div', recursive=False)[1].find_all('table')[1].find_all('tr')
persons = map(lambda x: x.find('span', class_='fio').text, persons)
persons = list(map(str.strip, persons))



In [ ]:
with open('files/persons.txt', 'w', encoding='utf-8') as f:
    json.dump({'persons': persons}, f, ensure_ascii=False)

In [22]:
cookie = '_ym_uid=1723625929764409548; _ym_d=1741671054; __utma=216042306.833214236.1741671054.1741767536.1741781916.3; _ym_isad=2; 216042306.1741671054.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); SUserID=604797893; SCookieGUID=6FFBB8F5%2DD107%2D401B%2DAB04%2D81BB3FBE7DB9; __utmc=216042306; __utmb=216042306.2.10.1741781916; __utmz=216042306.1741781916.3.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmt=1'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Cookie': cookie}

In [3]:
authors = []

In [3]:
import time

In [4]:
def check_captcha(page):
    return not page.find('div', class_='g-recaptcha') is None

In [ ]:
#find author on elibrary and get elibrary id
for person in persons:
    url = f"https://www.elibrary.ru/authors.asp?codetype=SPIN&sortorder=2&metrics=1&orgid=198&surname={person}&order=0"
    print(f"req {time.ctime()} {person}")
    response = requests.get(url, headers=headers)
    page = BeautifulSoup(response.text, 'lxml')

    if check_captcha(page):
        print(f'{time.ctime()} Captcha')
        break
        time.sleep(60)

    result = page.find('table', id='restab')
    if not result is None:
        author = result.find_all('tr', recursive=False)[3:][0].get('id')
        authors.append(author)        
        with open('files/authors.txt', 'a', encoding='utf-8') as f:
            f.write(f'{person}:{author}\n')

    persons = persons[1:]
    time.sleep(20)

req Wed Mar 12 11:28:01 2025 aaaa


In [ ]:
with open('files/authors.txt', 'r', encoding='utf-8') as f:
    authors = list(map(lambda x: x.split(':'), [line.rstrip() for line in f]))

authors_json = {'authors' : {}}
for author in authors:
    authors_json['authors'][author[1]] = {}
    authors_json['authors'][author[1]]['name'] = author[0]

In [23]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Cookie': cookie}

In [24]:
authors_json

{'authors': {'a1189828': {'name': 'Аббясов Булат Ривалевич',
   'publications': []},
  'a521475': {'name': 'Абрамский Михаил Михайлович'},
  'a825443': {'name': 'Агафонов Александр Алексеевич'},
  'a1146602': {'name': 'Алимова Ильсеяр Салимовна'},
  'a1236949': {'name': 'Апурин Артем Александрович'},
  'a1209572': {'name': 'Большаков Эдуард Сергеевич'},
  'a1217686': {'name': 'Валишин Искандер Айратович'},
  'a646549': {'name': 'Галлямова Динара Хамитовна'},
  'a295287': {'name': 'Гарипов Ильнур Бурханович'},
  'a609739': {'name': 'Голицына Ирина Николаевна'},
  'a303678': {'name': 'Григорян Карен Альбертович'},
  'a305468': {'name': 'Гузельбаева Гузель Яхиевна'},
  'a111026': {'name': 'Гусенков Александр Михайлович'},
  'a1175220': {'name': 'Доброквашина Александра Сергеевна'},
  'a2521': {'name': 'Елизаров Александр Михайлович'},
  'a1210222': {'name': 'Еникеева Аделя Искандеровна'},
  'a1192611': {'name': 'Жукова Валерия Юрьевна'},
  'a1261831': {'name': 'Загиров Айдар Ильнурович'},

In [25]:
to_iterate = list(authors_json['authors'].keys())

In [ ]:
#get publications of found authors
import math

for author in to_iterate:
    authors_json['authors'][author]['publications'] = []
    print(f"req {time.ctime()} {authors_json['authors'][author]['name']}")
    url = f"https://www.elibrary.ru/author_items.asp?authorid={author[1:]}&pubrole=100&show_refs=1&show_option=0&order=1"
    response = requests.get(url, headers=headers)
    page = BeautifulSoup(response.text, 'lxml')

    if check_captcha(page):
        print(f'{time.ctime()} Captcha')
        break
    
    try:
        count = int(page.find('td', class_='redref').find('b').text)
        num_pages = math.ceil(count / 100)

    except Exception as e:
        break
    
    time.sleep(20)
    for i in range(1, num_pages + 1):
        if i != 1:
            url = f"https://www.elibrary.ru/author_items.asp?pubrole=100&authorid={author[1:]}&order=1&pagenum={i}"
            response = requests.get(url, headers=headers)
            page = BeautifulSoup(response.text, 'lxml')

            if check_captcha(page):
                print(f'{time.ctime()} Captcha')
                break

            time.sleep(20)

        trs = page.find('table', id='restab').find_all('tr')[3:]
        for t in trs:
            authors_json['authors'][author]['publications'].append({})
            authors_json['authors'][author]['publications'][-1]['id'] = t.get('id')
            authors_json['authors'][author]['publications'][-1]['name'] = ''
            authors_json['authors'][author]['publications'][-1]['journal'] = ''
            try:
                authors_json['authors'][author]['publications'][-1]['name'] = t.find_all('span')[1].text
                authors_json['authors'][author]['publications'][-1]['journal'] = t.find_all('td')[1].find_all('font')[1].text.replace('\xa0', ' ').replace('\r', '').replace('\n', '')
            except:
                pass
    
    if check_captcha(page):
        break
    
    to_iterate = to_iterate[1:]

req Wed Mar 12 15:49:26 2025 Хайруллин Альфред Фаридович
req Wed Mar 12 15:49:47 2025 Хафизов Мурад Рустэмович
req Wed Mar 12 15:50:08 2025 Цой Татьяна Григорьевна
req Wed Mar 12 15:50:29 2025 Чеботарева Эльвира Валерьевна
req Wed Mar 12 15:50:51 2025 Шахова Ирина Сергеевна
req Wed Mar 12 15:51:11 2025 Шемахин Александр Юрьевич
req Wed Mar 12 15:51:33 2025 Широкова Елена Александровна
req Wed Mar 12 15:51:55 2025 Шубин Алексей Витальевич


In [ ]:
with open('files/publications.txt', 'w', encoding='utf-8') as f:
    json.dump(authors_json, f, ensure_ascii=False)